In [201]:
import scipy.io
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import keras
import matplotlib.pyplot as plt


In [202]:
def DataPreprocessing(data, ff):

    fmat = {0.1:data['sbsl_PINN'][0][0], 0.08:data['sbsl_PINN'][1][0], 0.06:data['sbsl_PINN'][2][0], 0.04:data['sbsl_PINN'][3][0], 0.02:data['sbsl_PINN'][4][0], 0.01:data['sbsl_PINN'][5][0], 0.005:data['sbsl_PINN'][6][0], 0.0:data['sbsl_PINN'][7][0]}
    fval = [0.1  , 0.08 , 0.06 , 0.04 , 0.02 , 0.01 , 0.005, 0.000]
    i = ff
    N = fmat[i].shape[1]
    U = np.zeros((N,4))
    U[:,0] = data['eta']
    U[:,1] = fmat[i][0,:]
    U[:,2] = fmat[i][1,:]
    U[:,3] = fmat[i][2,:]
    #U[:,4] = torch.from_numpy(np.array([0.06]))

    D = np.array(data['D'])
    M = np.array(data['M'])
    u = np.array(data['ubar'])
    dMdx = np.array(data['dMdx'])
    dudx = np.array(data['dudx'])
    alpha = np.array(data['alp'])
    eta1 = np.array(data['eta'])   

    meanflow = np.zeros((D.shape[1],7))
    meanflow[:,0] = D
    meanflow[:,1] = M
    meanflow[:,2] = u
    meanflow[:,3] = dMdx
    meanflow[:,4] = dudx
    meanflow[:,5] = alpha
    meanflow[:,6] = eta1
    #meanflow = torch.tile(meanflow, (8,1))

    he = np.array([0.00])
    HE = np.tile(he, (N,1)) 
    #He = HE.flatten()[:,None]

    uu = U[:, 1:4]
    uu0 = U[:,0]
    uu0 = uu0.flatten()[:, None]

    input_set = np.concatenate([HE, uu0],1)

    return input_set, uu, meanflow

In [203]:
data = scipy.io.loadmat('/Users/ramtarun/Desktop/Cambridge/Indirect-Noise-in-Nozzles/Data/Data_PINN_subsonic_geom_linvelsup_f0-0.1.mat')
inputs, targets, meanflow = DataPreprocessing(data, ff=0.01)

In [204]:

def split_data(U, b_size, n_batches):
    '''
    Splits the data in batches. Each batch is created by sampling the signal with interval
    equal to n_batches.
    '''
    data   = np.zeros((n_batches, b_size, U.shape[1]), dtype=float)    
    for j in range(n_batches):
        data[j,:b_size] = U[::skip][j::n_batches].copy()

    return data

In [205]:
skip = 2
b_size = 30
n_batches = 5
val_batches = n_batches//2   #validation batches

## Training Data

In [206]:
x_tt = inputs[:b_size*n_batches*skip].copy()
y_tt = targets[:b_size*n_batches*skip].copy()
m_tt = meanflow[:b_size*n_batches*skip].copy()

Y_train     = split_data(y_tt, b_size, n_batches).astype(dtype=np.float32)
X_train     = split_data(x_tt, b_size, n_batches).astype(dtype=np.float32)
Meanflow_train = split_data(m_tt, b_size, n_batches).astype(dtype=np.float32)

## Validation Data 

In [207]:
x_vv        = inputs[b_size*n_batches*skip:b_size*n_batches*skip+b_size*val_batches*skip:].copy()
Y_vv        = targets[b_size*n_batches*skip:b_size*n_batches*skip+b_size*val_batches*skip:].copy()
meanflow_vv = meanflow[b_size*n_batches*skip:b_size*n_batches*skip+b_size*val_batches*skip:].copy()
Y_val       = split_data(Y_vv, b_size, val_batches).astype(dtype=np.float32) 
X_val       = split_data(x_vv, b_size, val_batches).astype(dtype=np.float32)
Meanflow_val = split_data(meanflow_vv, b_size, val_batches).astype(dtype=np.float32)

In [208]:
def RHS_ff_t(x,y, baseflow, f):
    eta = x
    pi_p, pi_m, sig = tf.unstack(y,axis=1)

    D = baseflow[:,0]
    M = baseflow[:,1]
    u = baseflow[:,2]
    dMdx = baseflow[:,3]
    dudx = baseflow[:,4]
    alpha = baseflow[:,5]
    eta1 = baseflow[:,6]    
    
    He = 0#a constant input

    gamma = 1.4#constant
    
    Msq = tf.math.square(M)

    Lambda = 1 + Msq * (gamma-1)/2
    zeta = f*gamma*Msq - 2*tf.math.tan(alpha)
    C1= ((gamma - 1)*(1-Msq)*f)/(2*Lambda*zeta)
    Ca = -C1*M*u*dMdx*(2-(2*Msq/(1-Msq)) - (2*gamma*Msq*(-2*f*Lambda - (gamma-1)/gamma *zeta)/(2*Lambda*zeta)))
    Ff = -(dudx + (4*f*u/(2*D)))

    denom = (M**2*u - u + C1*Msq *u + C1*Msq*Msq*gamma*u) #M**4
    vrh_p = M*(2*(1-M) + C1*M*(M-2+M*gamma*(1-2*M)))/(2*denom)
    vrh_m = -M*(2*(1+M) + C1*M*(M+2+M*gamma*(1+2*M)))/(2*denom)
    vkp_p = Msq*C1*(2+M*(gamma-1))/(2*denom)
    vkp_m = Msq*C1*(2-M*(gamma-1))/(2*denom)
    vth_p = C1*M*(M*(1+gamma) + M**2*(1-gamma) - 2)/denom
    vth_m = C1*M*(M*(1+gamma) - M**2*(1-gamma) + 2)/denom
    vsig = -(C1*Msq*(1+gamma*Msq) + Msq - 1)/denom
    calM = dMdx/(2*M)
    kp_p = (gamma - 1) + (2/M)
    kp_m = (gamma - 1) - (2/M)
    Gm_p = M*(Ca*(M + 1) + Ff*M*(C1*gamma*M*Msq + M + (1 - C1*Msq)))/(2*denom)
    Gm_m = M*(Ca*(M - 1) - Ff*M*(C1*gamma*M*Msq + M - (1 - C1*Msq)))/(2*denom)
    Ups = (Ca*(Msq - 1) - C1*Ff*Msq*Msq *(1+gamma))/denom
    
#     eq1 = - (2*np.pi*1j*He*vrh_p + Gm_m*kp_m + calM)*pi_p + (2*np.pi*1j*He*vkp_p + Gm_m*kp_p + calM)*pi_m - Gm_m*sig
#     eq2 = - (2*np.pi*1j*He*vkp_m + Gm_p*kp_m - calM)*pi_p - (2*np.pi*1j*He*vrh_m + Gm_p*kp_p + calM)*pi_m - Gm_m*sig
#     eq3 = - (2*np.pi*1j*He*vth_p + Ups*kp_m)*pi_p - (2*np.pi*1j*He*vth_m + Ups*kp_p)*pi_m - (2*np.pi*1j*He*vsig + Ups)*sig
    
#     eq1 = - tf.multiply((Gm_m*kp_m + calM),pi_p) + tf.multiply(( Gm_m*kp_p + calM),pi_m) - tf.multiply(Gm_m,sig)
    eq1 =  (Gm_m*kp_m + calM)*pi_p + ( Gm_m*kp_p - calM)*pi_m + Gm_m*sig
    eq2 =  ( Gm_p*kp_m - calM)*pi_p + ( Gm_p*kp_p + calM)*pi_m + Gm_m*sig
    eq3 =  ( Ups*kp_m)*pi_p + ( Ups*kp_p)*pi_m + (Ups)*sig
    



    return tf.stack([-eq1, -eq2, -eq3],axis=1)

In [209]:
@tf.function
def pde(r, n, baseflow, f):
    dr_dn = tf.gradients(r,n)
    gr = RHS_ff_t(n, r, baseflow, f)
    pde = dr_dn  + gr
    return pde


In [210]:
@tf.function
def loss(outs, targets, pde):
    l1 = tf.keras.losses.MSE(targets, outs)
    l2 = tf.reduce_mean(pde**2)
    return l1+l2

In [211]:
### Model
model = keras.Sequential(name='PhyLSTM')
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True,input_shape=(None, 2)))
model.add(keras.layers.Dense(3, activation='sigmoid')) 

model.predict(X_train)

2023-07-21 20:59:50.471576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-21 20:59:50.628047: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_14/lstm_cell/kernel/Assign' id:12281 op device:{requested: '', assigned: ''} def:{{{node lstm_14/lstm_cell/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_14/lstm_cell/kernel, lstm_14/lstm_cell/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-07-21 20:59:52.004852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-21 20:59:52.374985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_

array([[[0.5       , 0.5       , 0.5       ],
        [0.50012755, 0.50005364, 0.50008917],
        [0.5003494 , 0.5001536 , 0.50025094],
        [0.50064117, 0.5002918 , 0.50047064],
        [0.50098455, 0.50046057, 0.50073576],
        [0.50136584, 0.50065273, 0.501036  ],
        [0.50177443, 0.5008623 , 0.50136274],
        [0.5022025 , 0.50108445, 0.5017094 ],
        [0.50264394, 0.5013151 , 0.5020705 ],
        [0.5030944 , 0.50155145, 0.50244206],
        [0.5035504 , 0.5017912 , 0.50282085],
        [0.5040097 , 0.5020327 , 0.50320446],
        [0.5044706 , 0.50227505, 0.50359106],
        [0.5049319 , 0.5025174 , 0.5039794 ],
        [0.50539297, 0.5027594 , 0.50436854],
        [0.5058532 , 0.50300086, 0.5047578 ],
        [0.5063124 , 0.50324154, 0.50514674],
        [0.5067704 , 0.50348157, 0.50553507],
        [0.5072271 , 0.50372094, 0.5059225 ],
        [0.5076827 , 0.5039599 , 0.50630903],
        [0.50813717, 0.5041984 , 0.5066945 ],
        [0.5085907 , 0.5044365 , 0

In [212]:
optim = tf.keras.optimizers.legacy.Adam(amsgrad=True)

In [213]:
@tf.function
def train(Xtrain, ytrain, baseflow, model, train = True):
    
    n = Xtrain[:,1]
    xtrain = tf.expand_dims(Xtrain, axis=1)
    outs = model(xtrain) [:,:3]
    f = tf.reduce_mean(outs[:,-1])
    print(f)
    PDE = pde(outs, n, baseflow, f)
    print(PDE)
    trn_loss = loss(outs, ytrain, pde)
    

    if train:
        param = model.trainable_weights + [f]
                #compute and apply gradients
        grads   = tf.gradients(loss, param)
        # grads   = tf.gradients(loss, varss, grad_ys= tf.complex128)
        optim.apply_gradients(zip(grads, param))


    return outs, trn_loss


In [214]:
plt.rcParams["figure.figsize"] = (15,4)
plt.rcParams["font.size"]  = 20

Loss_Mse    = tf.keras.losses.MeanSquaredError()

n_epochs    = 5001 #number of epochs

#define optimizer and initial learning rate   
optimizer   = tf.keras.optimizers.legacy.Adam(amsgrad=True) #amsgrad True for better convergence
# optimizer   = tf.keras.optimizers.Adam #amsgrad True for better convergence

l_rate                  = 0.01
optimizer.learning_rate = l_rate

# quantities to check and store the training and validation loss and the training goes on
old_loss      = np.zeros(n_epochs) #needed to evaluate training loss convergence
tloss_plot    = np.zeros(n_epochs) #training loss
vloss_plot    = np.zeros(n_epochs) #validation loss
tloss1_plot   = np.zeros(n_epochs) #training_der loss
vloss1_plot   = np.zeros(n_epochs) #validation_der loss
old_loss[0]  = 1e6 #initial value has to be high
N_check      = 5   #each N_check epochs we check convergence and validation loss
patience     = 200 #if the val_loss has not gone down in the last patience epochs, early stop
last_save    = patience

t            = 1 # initial (not important value) to monitor the time of the training

for epoch in range(n_epochs):
    
    if epoch - last_save > patience: break #early stop
                
    #Perform gradient descent for all the batches every epoch
    loss_training = 0
#     rng.shuffle(t_train, axis=0) #shuffle batches

    for j in range(n_batches-2):
        outs, loss  = train(X_train[j], Y_train[j], Meanflow_train[j], model)
        loss_training += loss
    
#     #save training loss each epoch
   
    tloss_plot[epoch]  = loss_training/n_batches
        
    if (epoch%100==0) and epoch != 0: 
        f = tf.mean(outs[:,-1])
        print(f.numpy())
        #plot convergence of training and validation loss
        plt.subplot(1,2,1)
        plt.title('MSE convergence')
        plt.yscale('log')
        plt.grid(True, axis="both", which='both', ls="-", alpha=0.3)
        plt.plot(tloss_plot[np.nonzero(tloss_plot)], 'g', label='Train loss')

        plt.xlabel('epochs')
        plt.subplot(1,2,2)
        plt.plot(Y_train[-1], 'black')
        plt.plot(outs[:,:3].numpy(), 'b--')

        
        plt.tight_layout()
        plt.show()

Tensor("Mean:0", shape=(), dtype=float32)


ValueError: in user code:

    File "/var/folders/rc/z9d7mxns5jgd6ggy35fswxnw0000gn/T/ipykernel_20304/1775593817.py", line 9, in train  *
        PDE = pde(outs, n, baseflow, f)
    File "/var/folders/rc/z9d7mxns5jgd6ggy35fswxnw0000gn/T/ipykernel_20304/2385997753.py", line 4, in pde  *
        gr = RHS_ff_t(n, r, baseflow, f)
    File "/var/folders/rc/z9d7mxns5jgd6ggy35fswxnw0000gn/T/ipykernel_20304/3186833219.py", line 3, in RHS_ff_t  *
        pi_p, pi_m, sig = tf.unstack(y,axis=1)

    ValueError: not enough values to unpack (expected 3, got 1)
